In [1]:
import numpy as np
import pyvista as pv

import structgeo.model as geo
import structgeo.generation as gen
import structgeo.plot as geovis


pv.set_jupyter_backend('static')
WS = (600,400) # Set a custom window size to be reused in most plots


## GeoProcess Icon Sketchbook

### Custom Plot Type for GeoProcess Icon Sketchbook 

In [2]:
def demoview(model, threshold=-0.5, opacity=1, plotter=None):

    if plotter is None:
        plotter = pv.Plotter(window_size=WS)
    p=plotter
    
    cats = np.arange(11)   # Set the number of categories for the colormap
    clim = [cats.min(), cats.max()] # Preset color limits for all subplots
    
    grid = geovis.get_voxel_grid_from_model(model, threshold=threshold)
    skin=grid.extract_surface()
    
    p.add_mesh(skin,scalars='values',clim=clim,cmap='gist_ncar',opacity=opacity)
    # p.add_axes(line_width=5)    
    p.remove_scalar_bar()
    
    flat_bounds = [item for sublist in model.bounds for item in sublist]
    bounding_box = pv.Box(flat_bounds)
    p.add_mesh(bounding_box, color="black", style="wireframe", line_width=3)
    p.camera.elevation = -20
    return p

SCALING = 2.0


### Layer Demo

In [3]:
bounds = ((-20, 20), (-20, 20), (-10, 10))
resolution = (128, 128, 64)
model = geo.GeoModel(bounds=bounds, resolution=resolution)

In [4]:
layer = geo.Layer(base=-10, width=10, value=0)
model.clear_history()
model.add_history(layer)
model.compute_model()
p = demoview(model)
p.screenshot("images/layer1.png", scale=SCALING, transparent_background=True)

layer = geo.Layer(base=0, width=5, value=2)
model.clear_history()
model.add_history(layer)
model.compute_model()
p = demoview(model, opacity=.8, plotter=p)

p.screenshot("images/layer2.png", scale=SCALING, transparent_background=True)
print('')

### Shift Demo


In [5]:
bedrock = geo.Bedrock(base=-10, value=0)
sb = geo.SedimentBuilder(1,10,4,4)
vals, thicks = sb.build_layers()
sediment = geo.Sedimentation(value_list=vals, thickness_list=thicks)
fold = geo.Fold(strike=30, dip=90, rake=0, origin=(0, 0, 0), amplitude=4, period=90, shape=1)
dike = geo.Dike(strike=45, dip=90, width=4, origin=(0, 0, 0), value=6)

model.clear_history()
model.add_history([bedrock,sediment, fold, dike])
model.compute_model()
p = demoview(model)

p.screenshot("images/shift1.png", scale=SCALING, transparent_background=True)

shift = geo.Shift(vector=(8,-4,8))
model.add_history(shift)
model.compute_model()
p = demoview(model)

p.screenshot("images/shift2.png", scale=SCALING, transparent_background=True);

### Rotate Demo

In [6]:
def simple_hist():
    bedrock = geo.Bedrock(base=-10, value=0)
    sb = geo.SedimentBuilder(1,10,4,4)
    vals, thicks = sb.build_layers()
    sediment = geo.Sedimentation(value_list=vals, thickness_list=thicks)
    fold = geo.Fold(strike=30, dip=90, rake=0, origin=(0, 0, 0), amplitude=4, period=90, shape=1)
    dike = geo.Dike(strike=45, dip=90, width=4, origin=(0, 0, 0), value=6)
    return [bedrock, sediment, fold, dike]


model.clear_history()
model.add_history(simple_hist())
model.compute_model()
p = demoview(model)

p.screenshot("images/rotate1.png", scale=SCALING, transparent_background=True)
axis = [0,0,1]
rot = geo.Rotate(axis=axis, angle=-30)
model.add_history(rot)
model.compute_model()
p = demoview(model)

p.screenshot("images/rotate2.png", scale=SCALING, transparent_background=True);

### Bedrock Demo

In [7]:
model.clear_history()
model.add_history(geo.NullProcess())
model.compute_model()
model.data[0] = 0 # Hack to bypass error warnings
p = demoview(model)
p.screenshot("images/bedrock1.png", scale=SCALING, transparent_background=True)

bedrock = geo.Bedrock(base=-2, value=0)
model.clear_history()
model.add_history(bedrock)
model.compute_model()
p = demoview(model)
p.screenshot("images/bedrock2.png", scale=SCALING, transparent_background=True);

### Sedimentation Demo

In [40]:
model.clear_history()
model.add_history(simple_hist())
model.compute_model()
p = demoview(model)
p.screenshot("images/sediment1.png", scale=SCALING, transparent_background=True)

sediment2 = geo.Sedimentation(value_list=[5,9,7,8], thickness_list=[1,3,2,3])
model.add_history(sediment2)
model.compute_model()
p = demoview(model, plotter=p, opacity=0.7)
p.screenshot("images/sediment2.png", scale=SCALING, transparent_background=True);

### Unconformity Demo

In [38]:
model.clear_history()
model.add_history(simple_hist())
model.compute_model()
p = demoview(model)
p.screenshot("images/unconf1.png", scale=SCALING, transparent_background=True)

unconformity = geo.UnconformityBase(-5)
model.add_history(unconformity)
model.compute_model()
p = demoview(model)
model.clear_history()
model.add_history(simple_hist())
model.compute_model()
p = demoview(model, plotter=p, opacity=0.2)
p.screenshot("images/unconfbase.png", scale=SCALING, transparent_background=True);


### Tilt Demo

In [39]:
model.clear_history()
model.add_history(simple_hist())
model.compute_model()
p = demoview(model)
p.screenshot("images/tilt1.png", scale=SCALING, transparent_background=True)

tilt = geo.Tilt(strike=-45, dip=15, origin= (20,20,0))
model.add_history(tilt)
model.compute_model()
p = demoview(model)
p.screenshot("images/tilt2.png", scale=SCALING, transparent_background=True);

### Dike Intrusions Demo

In [ ]:
def flat_hist():
    bedrock = geo.Bedrock(base=-10, value=0)
    sb = geo.SedimentBuilder(1,10,4,4)
    vals, thicks = sb.build_layers()
    sediment = geo.Sedimentation(value_list=vals, thickness_list=thicks)
    return [bedrock, dike]


model.clear_history()
model.add_history(flat_hist())
model.compute_model()
p = demoview(model)
p.screenshot("images/tilt1.png", scale=SCALING, transparent_background=True)

tilt = geo.Tilt(strike=-45, dip=15, origin= (20,20,0))
model.add_history(tilt)
model.compute_model()
p = demoview(model)
p.screenshot("images/tilt2.png", scale=SCALING, transparent_background=True);